In [3]:
import sys
import os
from IPython.display import Markdown
from env_key_manager import APIKeyManager

# 创建实例
key_manager = APIKeyManager()

# 设置环境变量
key_manager.setup_api_key(["DEEPSEEK_API_KEY", "LANGSMITH_ENDPOINT", "LANGSMITH_API_KEY", "LANGSMITH_PROJECT", "TAVILY_API_KEY"])

os.environ['LANGSMITH_TRACING'] = "true"

# 查看Python版本
!python -V
# 查看安装的库
if 'win' in sys.platform.lower():
    !pip list | findstr "lang openai llm tiktoken chromadb cryptography duck unstructured numpy scipy"
else:
    !pip list | grep -E "lang|openai|llm|tiktoken|chromadb|cryptography|duck|unstructured|numpy|scipy"

TAVILY_API_KEY密钥已加密保存
Python 3.10.16
chromadb                                 0.6.3
cryptography                             44.0.2
duckduckgo_search                        6.3.7
langchain                                0.3.19
langchain-chroma                         0.2.2
langchain-community                      0.3.18
langchain-core                           0.3.49
langchain-deepseek                       0.1.3
langchain-openai                         0.3.11
langchain-text-splitters                 0.3.6
langgraph                                0.3.21
langgraph-checkpoint                     2.0.23
langgraph-prebuilt                       0.1.7
langgraph-sdk                            0.1.60
langserve                                0.3.1
langsmith                                0.3.8
numpy                                    1.26.4
openai                                   1.69.0
scipy                                    1.15.2
tiktoken                                 0.9.0


# 构建一个智能体

语言模型本身无法采取行动——它们仅能输出文本。
LangChain 的一个重要应用场景是构建**智能代理**。
[智能体](/docs/concepts/agents)是使用[大语言模型](/docs/concepts/chat_models)作为推理引擎的系统，用于确定需要执行的操作以及执行这些操作所需的输入。
在执行操作后，可以将结果反馈给LLM（大语言模型），以判断是否需要进一步操作，或是可以结束流程。这一机制通常通过[工具调用](/docs/concepts/tool_calling)来实现。

在本教程中，我们将构建一个能够与搜索引擎交互的智能体。你可以向这个智能体提问，观察它调用搜索工具的过程，并与之进行对话交流。

## 端到端代理

以下代码片段展示了一个功能完备的智能体，该智能体利用大语言模型（LLM）来决定使用哪些工具。它配备了一个通用搜索工具，并具有对话记忆功能——这意味着它可以作为多轮对话的聊天机器人使用。

在本指南的剩余部分，我们将逐步讲解各个组件及其功能——但如果您想直接获取代码并立即开始，请随意使用以下内容！

In [4]:
# 导入相关功能
from langchain_deepseek import ChatDeepSeek
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# 创建代理
memory = MemorySaver()  # 保存对话历史
model = ChatDeepSeek(model_name="deepseek-chat") # 深度搜索模型
search = TavilySearchResults(max_results=2) # 搜索结果
tools = [search] # 工具
agent_executor = create_react_agent(model, tools, checkpointer=memory) # 创建代理

In [6]:
# 使用代理
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="嗨, 我是Bob! 我住在sf")]},
    config,
    stream_mode="values", # "values" or "steps"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

嗨, 我是Bob! 我住在sf
================================== Ai Message ==================================

嗨，Bob！又见面啦！旧金山（SF）是个充满活力的城市，你最近有没有发现什么新鲜有趣的地方或活动？或者有什么关于SF的问题想聊聊吗？ 😄


In [7]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_0_3ba4af33-5f04-4799-9ae0-a46c4711ed31)
 Call ID: call_0_3ba4af33-5f04-4799-9ae0-a46c4711ed31
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1743235847, 'localtime': '2025-03-29 01:10'}, 'current': {'last_updated_epoch': 1743235200, 'last_updated': '2025-03-29 01:00', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/nig

## 安装设置

### Jupyter 笔记本

本指南（以及文档中的大多数其他指南）使用 [Jupyter Notebook](https://jupyter.org/) 并假设读者也在使用该工具。Jupyter Notebook 是学习如何构建 LLM 系统的理想交互环境，因为过程中常会出现意外情况（如非预期输出、API 宕机等），而观察这些案例能有效帮助开发者深入理解 LLM 开发。

本教程及其他教程或许在 Jupyter notebook 中运行最为便捷。有关安装说明，请参阅[此处](https://jupyter.org/install)。

### 安装

要安装LangChain，请运行：

In [ ]:
# %pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

更多详情，请参阅我们的[安装指南](/docs/how_to/installation)。

### LangSmith

使用LangChain构建的许多应用程序将包含多个步骤，涉及多次LLM调用。
随着这些应用变得越来越复杂，能够检查链或代理内部的具体运行情况变得至关重要。
最佳方式是使用 [LangSmith](https://smith.langchain.com)。

在以上链接完成注册后，请确保设置环境变量以开始记录追踪数据：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

```bash
export TAVILY_API_KEY="..."
```
```python
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()
```

## 定义工具

我们首先需要创建想要使用的工具。我们选择的主要工具将是[Tavily](/docs/integrations/tools/tavily_search)——一个搜索引擎。LangChain内置了一个工具，可以轻松将Tavily搜索引擎作为工具使用。

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1739993250, 'localtime': '2025-02-19 11:27'}, 'current': {'last_updated_epoch': 1739992500, 'last_updated': '2025-02-19 11:15', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png', 'code': 1183}, 'wind_mph': 5.8, 'wind_kph': 9.4, 'wind_degree': 195, 'wind_dir': 'SSW', 'pressure_mb': 1023.0, 'pressure_in': 30.2, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 100, 'feelslike_c': 12.7, 'feelslike_f': 54.8, 'windchill_c': 9.1, 'windchill_f': 48.4, 'heatindex_c': 10.2, 'heatindex_f': 50.3, 'dewpoint_c': 9.8, 'dewpoint_f': 49.7, 'vis_km': 4.0, 'vis_miles': 2.0, 'uv': 1.4, 'gust_mph': 8.9, 'gust_kph': 14.4}}"}, {'url': 'https://weathershogun.com/weather/usa/ca/

## 使用语言模型

接下来，让我们学习如何使用语言模型调用工具。LangChain支持多种可互换使用的语言模型——请从下方选择您想使用的模型！

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4"}}} />


In [8]:
# | output: false
# | echo: false

from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(model="deepseek-chat")

你可以通过传入消息列表来调用语言模型。默认情况下，响应是一个 `content` 字符串。

In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
Markdown(response.content)

'Hello! 😊 How can I help you today?'

现在我们可以看到让这个模型执行工具调用的效果。为了启用这一功能，我们使用`.bind_tools`方法让语言模型了解这些工具。

In [10]:
model_with_tools = model.bind_tools(tools)

我们现在可以调用模型了。首先用一个普通消息来调用它，看看它是如何响应的。我们可以同时查看 `content` 字段和 `tool_calls` 字段。

In [11]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today? 😊
ToolCalls: []


现在，让我们尝试用一些预期会调用工具的输入来调用它。

In [12]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_0_6f422c6c-724b-4717-83f6-c2bc70336895', 'type': 'tool_call'}]


我们可以看到现在没有文本内容，但有一个工具调用！它希望我们调用Tavily搜索工具。

这还不是在调用那个工具——它只是在指示我们这么做。为了真正调用它，我们需要创建自己的代理。

## 创建代理

既然我们已经定义了工具和LLM，现在可以创建代理了。我们将使用[LangGraph](/docs/concepts/architecture/#langgraph)来构建这个代理。
目前，我们正使用高级接口来构建智能体，但LangGraph的优势在于：这一高级接口背后由一套低层级、高度可控的API支撑，方便您随时调整智能体逻辑。

现在，我们可以用LLM和工具来初始化代理。

请注意，我们传入的是 `model` 而非 `model_with_tools`。这是因为 `create_react_agent` 会在底层自动为我们调用 `.bind_tools` 方法。

In [14]:
from langgraph.prebuilt import create_react_agent

# 创建代理
agent_executor = create_react_agent(model, tools)

## 运行代理

我们现在可以运行代理来处理一些查询了！请注意，目前这些查询都是**无状态**的（它不会记住之前的交互）。同时要注意，代理会在交互结束时返回**最终**状态（其中包含所有输入内容，稍后我们将介绍如何仅获取输出部分）。

首先，让我们看看在无需调用工具时它的反应如何：

In [15]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='c0a025b5-3339-4bd7-ad09-2fb66ad42dd7'),
 AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 146, 'total_tokens': 157, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 18}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': '1a8465c8-d90d-4286-a035-eb0663235616', 'finish_reason': 'stop', 'logprobs': None}, id='run-166e5980-604f-426b-b1e1-d70d412e5210-0', usage_metadata={'input_tokens': 146, 'output_tokens': 11, 'total_tokens': 157, 'input_token_details': {'cache_read': 128}, 'output_token_details': {}})]

为了准确了解底层运行机制（并确保其未调用工具），我们可以查看 [LangSmith 追踪记录](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)。

现在让我们在一个应该调用工具的例子中尝试一下

In [11]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='1d6c96bb-4ddb-415c-a579-a07d5264de0d'),
 AIMessage(content=[{'id': 'toolu_01Y5EK4bw2LqsQXeaUv8iueF', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_0132wQUcEduJ8UKVVVqwJzM4', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 269, 'output_tokens': 61}}, id='run-26d5e5e8-d4fd-46d2-a197-87b95b10e823-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in san francisco'}, 'id': 'toolu_01Y5EK4bw2LqsQXeaUv8iueF'}], usage_metadata={'input_tokens': 269, 'output_tokens': 61, 'total_tokens': 330}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localti

我们可以查看 [LangSmith 追踪记录](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) 来确认它是否有效调用了搜索工具。

## 流式消息

我们已经了解了如何通过`.invoke`调用智能体来获取最终响应。如果智能体执行多个步骤，这可能需要一些时间。为了展示中间进度，我们可以在消息发生时将其流式传输回来。

In [14]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================

[{'text': 'Okay, let me look up the current weather for San Francisco using a search engine:', 'type': 'text'}, {'id': 'toolu_01H1brh5EZpZqtqHBxkosPtN', 'input': {'query': 'san francisco weather'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01H1brh5EZpZqtqHBxkosPtN)
 Call ID: toolu_01H1brh5EZpZqtqHBxkosPtN
  Args:
    query: san francisco weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1739994486, 'localtime': '2025-02-19 11:48'},

## 流式令牌

除了流式传回消息外，流式传回令牌也很有用。
我们可以通过指定 `stream_mode="messages"` 来实现这一点。


::: 注意

以下我们使用 `message.text()`，这需要 `langchain-core>=0.3.37`。

:::

In [21]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")



Base|d on the weather| search| results, here| are the key details| about the weather in| San Francisco:|

- The current temperature| in| San Francisco is aroun|d 55|-|56|°F (13|°|C).| Light| rain is occurring with| |100|% clou|d cover. |

-| Winds| are aroun|d 5-9| mph from| the south|-southwest.|

- The| forecast| for| the rest| of February| 2025 |shows da|ytime highs mostly| in the upper| 50s to| low| 60s°|F,| with overnight lows| in| the upper| 40s to| low| 50s°|F.|

-| Overall|, typical| cool| an|d show|ery late| winter weather is| expected in San Francisco| for the remainder| of February,| with a| mix| of rain| and dry| periods|.| Temperatures will be| season|able| for| this| time of year.|

So| in summary, San| Francisco is| experiencing light| rain an|d cool| temperatures currently, but| the late| winter forecast| shows typical mil|d and show|ery conditions| pers|isting through the en|d of the| month.| Let| me know if you| need any other| details about| the weather in the| cit

## 添加到内存中

如前所述，该代理是无状态的。这意味着它不会记住之前的交互。为了赋予其记忆能力，我们需要传入一个检查点记录器。当传入检查点记录器时，在调用代理时还必须传入一个`thread_id`（以便它知道从哪个线程/对话恢复）。

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [12]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [13]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! It's nice to meet you again.", response_metadata={'id': 'msg_013C1z2ZySagEFwmU1EsysR2', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 1162, 'output_tokens': 14}}, id='run-f878acfd-d195-44e8-9166-e2796317e3f8-0', usage_metadata={'input_tokens': 1162, 'output_tokens': 14, 'total_tokens': 1176})]}}
----


In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You mentioned your name is Bob when you introduced yourself earlier. So your name is Bob.', response_metadata={'id': 'msg_01WNwnRNGwGDRw6vRdivt6i1', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 1184, 'output_tokens': 21}}, id='run-f5c0b957-8878-405a-9d4b-a7cd38efe81f-0', usage_metadata={'input_tokens': 1184, 'output_tokens': 21, 'total_tokens': 1205})]}}
----


示例 [LangSmith 跟踪记录](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

如果你想开启一个新对话，只需更改所使用的 `thread_id` 即可

In [15]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm afraid I don't actually know your name. As an AI assistant without personal information about you, I don't have a specific name associated with our conversation.", response_metadata={'id': 'msg_01NoaXNNYZKSoBncPcLkdcbo', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 267, 'output_tokens': 36}}, id='run-c9f7df3d-525a-4d8f-bbcf-a5b4a5d2e4b0-0', usage_metadata={'input_tokens': 267, 'output_tokens': 36, 'total_tokens': 303})]}}
----


## 结论

搞定！在这个快速入门指南中，我们介绍了如何创建一个简单的智能体。
随后我们展示了如何流式返回响应——不仅包含中间步骤，还包括令牌信息！
我们还添加了记忆功能，这样你就可以与他们进行对话了。
代理是一个复杂的话题，涉及大量知识需要学习！

如需了解更多关于智能体的信息，请查阅 [LangGraph](/docs/concepts/architecture/#langgraph) 文档。该文档包含专属的概念解析、教程指南及操作说明。